In [2]:
from torchsummary import summary
from python_lib.modules import *
import torch
import numpy as np
import os

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [3]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")
input_feats = torch.rand([10,2,64]).to("cuda")
output = model(input_feats)
output.shape

torch.Size([10, 6])

# Entire ECAPA-TDNN Model overview

In [4]:
# Default Activation fn = Relu and group = 1

initialblock = TDNNBlock(in_channels=2,
                        out_channels=8,
                        kernel_size=5,
                        dilation= 1).to("cuda")

x0 = initialblock(input_feats)
print("Initial out: ",x0.shape)

seres2_1 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=2).to("cuda")

x1 = seres2_1(x0)
print("SERes2_1 out: ",x1.shape)

seres2_2 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=3).to("cuda")

x2 = seres2_2(x1)
print("SERes2_2 out: ",x2.shape)

seres2_3 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=4).to("cuda")

x3 = seres2_3(x2)
print("SERes2_3 out: ",x3.shape)

mfa = TDNNBlock(in_channels=(8 * 3),
                out_channels=16,
                kernel_size=1,
                dilation= 1).to("cuda")

x4 = mfa(torch.cat([x1,x2,x3], dim=1))
print("mfa out: ",x3.shape)

asp = AttentiveStatisticsPooling(16, 128, True).to("cuda")
x5 = asp(x4)
x6 = nn.BatchNorm1d(16 * 2).to("cuda")(x5)
print("asp out: ",x6.shape)

# Remember to x2 to account for the mean & std in asp
final = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16 * 2, out_features=6),
        ).to("cuda")

y = final(x6)
print("final out: ",y.shape)

Initial out:  torch.Size([10, 8, 64])
SERes2_1 out:  torch.Size([10, 8, 64])
SERes2_2 out:  torch.Size([10, 8, 64])
SERes2_3 out:  torch.Size([10, 8, 64])
mfa out:  torch.Size([10, 8, 64])
asp out:  torch.Size([10, 32, 1])
final out:  torch.Size([10, 6])


## Disecting SERes2NetBlock

Since in_channel == out_channel, shortcut is not necessary

In [5]:
tdnn1 = TDNNBlock(in_channels=8, 
                  out_channels=8, 
                  kernel_size=1, 
                  dilation=1).to("cuda")

z0 = tdnn1(x0)
print(z0.shape)
res2net = Res2NetBlock(in_channels=8, 
                       out_channels=8, 
                       scale=8, 
                       kernel_size=3, 
                       dilation=1).to("cuda")

z1 = res2net(z0)
print(z1.shape)

tdnn2 = TDNNBlock(in_channels=8, 
                  out_channels=8, 
                  kernel_size=1, 
                  dilation=1).to("cuda")

z2 = tdnn2(z1)
print(z2.shape)

se_block = SEBlock(in_channels=8, 
                   se_channels=128, 
                   out_channels=8).to("cuda")

z3 = se_block(z2)
print((z3 + x0).shape)


torch.Size([10, 8, 64])
torch.Size([10, 8, 64])
torch.Size([10, 8, 64])
torch.Size([10, 8, 64])


## Disecting Res2NetBlock

Here Scale = 8

In [6]:
scale = 8
in_channel = 8//scale # 1
hidden_channel = 8//scale # 1

block1 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block2 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block3 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block4 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block5 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block6 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block7 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")

chunks = torch.chunk(z0, 8, 1)
# for i in chunks:
#     print(i.shape)

b7 = block7(chunks[0])
print(chunks[0].shape)
print(b7.shape)

torch.Size([10, 1, 64])
torch.Size([10, 1, 64])


In [7]:
from speechbrain.nnet.CNN import Conv1d as _Conv1d

class Conv1d(_Conv1d):
    """1D convolution. Skip transpose is used to improve efficiency."""

    def __init__(self, *args, **kwargs):
        super().__init__(skip_transpose=True, *args, **kwargs)

c1 = Conv1d(in_channels=1,
            out_channels=1,
            kernel_size=3,
            dilation=1,
            groups=1).to("cuda")

c2 = nn.Conv1d(1,1,3,1,0,1).to("cuda")

print(c1(chunks[0]).shape)
print(c2(chunks[0]).shape)

torch.Size([10, 1, 64])
torch.Size([10, 1, 62])


In [8]:
c1.conv.weight

Parameter containing:
tensor([[[ 0.4432, -0.0911,  0.3460]]], device='cuda:0', requires_grad=True)

In [35]:
input_feats = torch.rand([1,1,64]).to("cuda")
input_feats_np = input_feats.cpu().detach().numpy()
input_feats_np = np.reshape(input_feats_np, (1,64))
print(input_feats_np)
dim = input_feats_np.shape

flatten_inputs = input_feats_np.flatten()
with open(os.path.join("ECAPAweights", f"inputseres2_1.bin"), "wb") as f:
            # Write the dimensions down
            f.write(np.array(dim, dtype=np.int32).tobytes())
            # Write the flatten bias down
            f.write(flatten_inputs.tobytes())

[[0.26740795 0.8178966  0.12961799 0.00493336 0.3306914  0.5511803
  0.62650675 0.27641416 0.7658161  0.4181484  0.32229787 0.48441684
  0.1843239  0.52775484 0.6218187  0.4034167  0.20479006 0.46894705
  0.7366815  0.6694186  0.05417478 0.2161662  0.9805711  0.29606843
  0.95383537 0.23102427 0.08317649 0.980294   0.82970244 0.6477569
  0.581959   0.0185324  0.7994169  0.37473667 0.24644166 0.19074416
  0.24360108 0.5712252  0.05087298 0.76147324 0.7151757  0.598813
  0.2791242  0.71931446 0.29900938 0.37861872 0.17825103 0.0577479
  0.6748667  0.23486334 0.6465089  0.1445471  0.3669144  0.7481721
  0.9296756  0.04023647 0.45330197 0.7755589  0.52666116 0.6326515
  0.9938532  0.01646823 0.10812438 0.52814806]]


In [36]:
import torch.nn.functional as F
input1 = F.pad(input_feats, (2,0))
input1

tensor([[[0.0000, 0.0000, 0.2674, 0.8179, 0.1296, 0.0049, 0.3307, 0.5512,
          0.6265, 0.2764, 0.7658, 0.4181, 0.3223, 0.4844, 0.1843, 0.5278,
          0.6218, 0.4034, 0.2048, 0.4689, 0.7367, 0.6694, 0.0542, 0.2162,
          0.9806, 0.2961, 0.9538, 0.2310, 0.0832, 0.9803, 0.8297, 0.6478,
          0.5820, 0.0185, 0.7994, 0.3747, 0.2464, 0.1907, 0.2436, 0.5712,
          0.0509, 0.7615, 0.7152, 0.5988, 0.2791, 0.7193, 0.2990, 0.3786,
          0.1783, 0.0577, 0.6749, 0.2349, 0.6465, 0.1445, 0.3669, 0.7482,
          0.9297, 0.0402, 0.4533, 0.7756, 0.5267, 0.6327, 0.9939, 0.0165,
          0.1081, 0.5281]]], device='cuda:0')

In [38]:
random_layer = torch.nn.Conv1d(1,1,3)

In [39]:
print(random_layer.weight)
print(random_layer.bias)

Parameter containing:
tensor([[[ 0.1354, -0.0436,  0.3382]]], requires_grad=True)
Parameter containing:
tensor([0.3302], requires_grad=True)


In [40]:
from python_lib.saveasfile import SaveAsByte

_ = SaveAsByte(random_layer, "test", "ECAPAweights")
_.saveBoth()

In [24]:
def banana(x):
    random_layer.to("cuda")
    y = random_layer(x)
    y = torch.nn.ReLU().to("cuda")(y)
    return torch.nn.BatchNorm1d(1).to("cuda")(y)

In [46]:
y = banana(input1.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)
y = F.pad(y, (2,0))
y = banana(y.to("cuda"))
print(y)

tensor([[[-1.0534,  0.7279, -1.5240, -0.8870, -0.6580, -0.2143,  0.3978,
          -0.5397,  1.4099, -0.4922,  0.0081,  0.1300, -1.1107,  0.4327,
           0.1897, -0.1316, -0.5900,  0.1086,  0.6408,  0.6543, -1.0698,
          -0.3294,  1.3872, -1.0926,  2.5396, -1.1948, -0.4742,  1.6899,
           0.5660,  1.2448,  0.8905, -1.2767,  1.5792, -1.0134, -0.1880,
          -0.9100, -0.8800,  0.1505, -1.7188,  1.4190,  0.2235,  0.8244,
          -0.2914,  1.2099, -0.8791,  0.1914, -1.1167, -1.3334,  0.5739,
          -1.3866,  1.0834, -1.4410,  0.1192,  0.6426,  1.4067, -1.2179,
           0.8552,  0.5545,  0.1225,  1.0447,  1.9005, -1.4882, -0.2372,
          -0.1787]]], device='cuda:0', grad_fn=<CudnnBatchNormBackward0>)
tensor([[[-1.0828,  0.7083, -1.0828, -0.5209, -1.0828, -0.6040,  0.0252,
          -0.7913,  1.4626, -0.9990,  0.4866, -0.1987, -1.0828,  0.4788,
          -0.4349, -0.1158, -0.6163, -0.0039,  0.2476,  0.4662, -1.0063,
          -0.0638,  0.8423, -1.0828,  3.0171, -1.0

In [59]:
x2 = torch.rand([1,3,5]).to("cuda")
print(x2.shape)
print(x2)
s = x2.mean(dim=2, keepdim=True)
print(s)
print(s.shape)

torch.Size([1, 3, 5])
tensor([[[0.2539, 0.5043, 0.8909, 0.5095, 0.1171],
         [0.0512, 0.3143, 0.4812, 0.7679, 0.6774],
         [0.9185, 0.1539, 0.9167, 0.9721, 0.0484]]], device='cuda:0')
tensor([[[0.4552],
         [0.4584],
         [0.6019]]], device='cuda:0')
torch.Size([1, 3, 1])


In [56]:
(-0.3841+0.6320+0.4254-0.7384)/4

-0.016274999999999984

In [62]:
se_block1 = SEBlock(in_channels=8, 
                   se_channels=128, 
                   out_channels=8).to("cuda")

x3 = torch.rand([1,8,5]).to("cuda")
se_block1(x3).shape

torch.Size([1, 8, 5])